In [ ]:
import os
import pandas as pd


# get files below path
def compile_file_list(path):
    file_list = []
    for root, _, files in os.walk(path):
        for name in files:
            file_list.append(os.path.join(path, root, name))

    # filter out non csvs
    file_list = [file for file in file_list if ".csv" in file]

    return file_list


# merge csvs into single df
def join_files_to_df(file_list, dup_drop=[]):
    df = pd.concat(map(pd.read_csv, file_list), ignore_index=True)
    if len(dup_drop) > 0:
        df = df.drop_duplicates(subset=dup_drop)
    return df

In [ ]:
account_files = compile_file_list(
    "/Users/lucasestrada/Projects/IMI/data_feeds_kit/feeds/accounts/"
)
account_df = join_files_to_df(account_files, ["account_id", "aws_account_id"])[["account_id", "aws_account_id", "mailing_address_id", "tax_address_id"]]

In [ ]:
product_files = compile_file_list(
    "/Users/lucasestrada/Projects/IMI/data_feeds_kit/feeds/product/"
)
product_df = join_files_to_df(product_files, ["product_id", "manufacturer_account_id"])[["product_id", "manufacturer_account_id", "title"]]

In [ ]:
agreement_files = compile_file_list(
    "/Users/lucasestrada/Projects/IMI/data_feeds_kit/feeds/agreements/"
)
agreement_df = join_files_to_df(agreement_files)[["proposer_account_id", "acceptor_account_id", "agreement_id", "origin_offer_id"]]

In [ ]:
df_account_merge = pd.merge(agreement_df, account_df, left_on="acceptor_account_id", right_on="account_id", how="inner").drop_duplicates(subset=['acceptor_account_id'])

In [ ]:
offer_product_files = compile_file_list(
    "/Users/lucasestrada/Projects/IMI/data_feeds_kit/feeds/offer_product/"
)
offer_product_df = join_files_to_df(
    offer_product_files, ["product_id", "offer_id", "offer_revision"]
)[["product_id", "offer_id"]]
offer_files = compile_file_list(
    "/Users/lucasestrada/Projects/IMI/data_feeds_kit/feeds/offer/"
)
offer_df = join_files_to_df(offer_files, ["offer_id", "offer_revision"])[["offer_id", "offer_revision"]]

df = pd.merge(product_df, offer_product_df, on="product_id", how="inner")
df2 = pd.merge(df, offer_df, on="offer_id", how="inner")
result = pd.merge(df2, df_account_merge, left_on="offer_id", right_on="origin_offer_id", how="inner").drop_duplicates(subset=['acceptor_account_id'])[["title", "aws_account_id"]]


In [ ]:
address_files = compile_file_list(
    "/Users/lucasestrada/Projects/IMI/data_feeds_kit/feeds/addresses/"
)
address_df = join_files_to_df(address_files)

In [ ]:
result2 = pd.merge(result, address_df, how="inner", on="aws_account_id")

In [ ]:
result2

In [3]:
import os
import pandas as pd
import boto3


# get files below path
def compile_file_list(path):
    file_list = []
    for root, _, files in os.walk(path):
        for name in files:
            file_list.append(os.path.join(root, name))

    # filter out non csvs
    file_list = [file for file in file_list if ".csv" in file]

    return file_list


# merge csvs into single df
def join_files_to_df(path, dup_drop=[]):
    file_list = compile_file_list(path)
    print(f"file_list: {file_list}")
    
    df = pd.concat(map(pd.read_csv, file_list), ignore_index=True)
    if len(dup_drop) > 0:
        df = df.drop_duplicates(subset=dup_drop)
    return df


import boto3
import os


def download_s3_folder(bucket_name, s3_folder, local_path):
    s3 = boto3.client("s3")

    # List all objects in the S3 folder
    objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_folder)

    # Recursively download each object
    for obj in objects.get("Contents", []):
        s3_key = obj["Key"]
        local_file_path = os.path.join(local_path, os.path.relpath(s3_key, s3_folder))

        # Skip directories
        if s3_key.endswith("/"):
            continue

        # Create local directory structure if needed
        local_dir = os.path.dirname(local_file_path)
        os.makedirs(local_dir, exist_ok=True)

        # Download the file
        s3.download_file(bucket_name, s3_key, local_file_path)

        print(f"Downloaded: s3://{bucket_name}/{s3_key} to {local_file_path}")


# extract user info from data feeds and join to product name
def get_user_info(base_path, product_name):
    print("base_path")
    print(base_path)
    tables = [
        "AccountFeed_V1",
        "AddressFeed_V1",
        "AgreementFeed",
        "OfferFeed_V1",
        "OfferProductFeed_V1",
        "ProductFeed_V1",
    ]
    df_dict = {}
    for table in tables:
        df_dict[table] = join_files_to_df(f"{base_path}/{table}/")

    df_dict["acct_ag"] = pd.merge(
        df_dict["AgreementFeed"],
        df_dict["AccountFeed_V1"],
        left_on="acceptor_account_id",
        right_on="account_id",
        how="inner",
    )

    df_dict["prod_off_prod"] = pd.merge(
        df_dict["ProductFeed_V1"], df_dict["OfferProductFeed_V1"], on="product_id", how="inner"
    )
    df_dict["prod_off"] = pd.merge(
        df_dict["prod_off_prod"], df_dict["OfferFeed_V1"], on="offer_id", how="inner"
    )
    intermediate = pd.merge(
        df_dict["prod_off"],
        df_dict["acct_ag"],
        left_on="offer_id",
        right_on="origin_offer_id",
        how="inner",
    )#[["title", "aws_account_id"]]
    return intermediate
    
bucket_name = "imi-subscribers"
s3_folder = ""
local_path = "feeds"
product_name = "Integrated Methane Inversion"
download_data_feeds = False

if download_data_feeds:
    download_s3_folder(bucket_name, s3_folder, local_path)
user_info = get_user_info(local_path, product_name)
# user_info.to_csv(f"user_info_{product_name.replace(' ', '_')}.csv", index=False)

base_path
feeds
file_list: ['feeds/AccountFeed_V1/year=2021/month=11/data.csv', 'feeds/AccountFeed_V1/year=2021/month=10/data.csv', 'feeds/AccountFeed_V1/year=2021/month=09/data.csv', 'feeds/AccountFeed_V1/year=2021/month=08/data.csv', 'feeds/AccountFeed_V1/year=2021/month=12/data.csv', 'feeds/AccountFeed_V1/year=2023/month=03/data.csv', 'feeds/AccountFeed_V1/year=2023/month=04/data.csv', 'feeds/AccountFeed_V1/year=2023/month=05/data.csv', 'feeds/AccountFeed_V1/year=2023/month=02/data.csv', 'feeds/AccountFeed_V1/year=2023/month=07/data.csv', 'feeds/AccountFeed_V1/year=2023/month=08/data.csv', 'feeds/AccountFeed_V1/year=2023/month=01/data.csv', 'feeds/AccountFeed_V1/year=2023/month=06/data.csv', 'feeds/AccountFeed_V1/year=2022/month=03/data.csv', 'feeds/AccountFeed_V1/year=2022/month=04/data.csv', 'feeds/AccountFeed_V1/year=2022/month=05/data.csv', 'feeds/AccountFeed_V1/year=2022/month=02/data.csv', 'feeds/AccountFeed_V1/year=2022/month=11/data.csv', 'feeds/AccountFeed_V1/year=2022/mont

In [4]:
user_info

,valid_from,insert_date,update_date,delete_date,address_id,email_domain,company_name,country_code,state_or_region,city,postal_code,address_line_1,address_line_2,address_line_3
0,2023-04-06T19:53:50Z,2023-06-22T07:26:34Z,2023-07-07T01:36:51Z,NaN,SgENQH66D1Beobh7XB/5p5PJ4tMvaPVnmWJRVUWC/DE=,duke.edu,"NSOE, Duke University",US,North Carolina,Durham,27705,2752 Campus Walk Ave,#24F,NaN
1,2022-10-19T12:22:16Z,2023-06-22T07:26:34Z,2023-07-07T01:36:51Z,NaN,4zgKsITq7BShD3VkrzBjtD8kj027cVsxSW7D/qXvD0U=,gmail.com,NaN,CN,beijing,beijing,100101,no11 datun road,NaN,NaN
2,2023-05-13T16:28:46Z,2023-06-22T07:26:34Z,2023-07-07T01:36:51Z,NaN,V2lf9I/PXl4zDfE+U4Itm2pX/+twY4gkpHG2B+bttIo=,gmail.com,NaN,DE,NaN,Aachen,52062,Komphausbadstrasse 2,NaN,NaN
3,2023-06-19T15:27:14Z,2023-06-22T07:26:34Z,2023-07-07T01:36:51Z,NaN,a2d1Uo+Q/ZmagsIQTuI0Q0+ziM/T3mPn4dtl56Tg2nw=,gmail.com,NaN,SE,Västerbotten,Umeå,90331,Häradshövdingegatan 8,NaN,NaN
4,2022-08-25T18:19:36Z,2023-06-22T07:26:34Z,2023-07-07T01:36:51Z,NaN,BXod1jFmM6fmbxtjeGhg+Zzec3+TU4W6WyLwgnlXR2E=,hardyland.ca,NaN,CA,NL,St. Johns,A1B 3X5,Morrissey Road,Captain Robert A. Bartlett Building,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2022-08-03T15:37:10Z,2023-06-22T07:26:34Z,2023-07-07T01:36:51Z,NaN,pQPE8xqDPJxkdPDcZBs89xP6x7EpaF4jlcLWNXq4O/c=,amazon.com,NaN,ZA,Western Cape,Cape Town,8001,Wembley Two,Solan Road,NaN
69,2023-03-07T02:39:48Z,2023-06-22T07:26:34Z,2023-07-07T01:36:51Z,NaN,cfa9K/f6tqKgl5RmZ6aRPxL03Fpwwd2dr2QYEKHJoco=,amazon.com,NaN,US,Texas,Frisco,75033,2455 Dillon Dr.,NaN,NaN
70,2023-07-14T01:16:51Z,2023-07-14T12:21:29Z,2023-07-14T12:21:29Z,NaN,6sM2QeD9h4trFVjTnaGoQt3wiOy/cZbe4z2SvTiQT6Q=,orbio.earth,Orbio Earth GmbH,DE,NRW,Köln,50735,Geestemünder Str. 41,NaN,NaN
71,2023-07-23T14:01:07Z,2023-07-24T12:11:43Z,2023-07-24T12:11:43Z,NaN,qs2SzIWX2vyPSwhB1I9Ms9OF0WY4Met/fAF3kXKlynk=,gmail.com,NaN,DE,NaN,Aachen,52062,Komphausbadstrasse 2,NaN,NaN


In [2]:
user_info

,title,aws_account_id
0,Integrated Methane Inversion,925689637756
1,Integrated Methane Inversion,925689637756
2,Integrated Methane Inversion,735873076892
3,Integrated Methane Inversion,735873076892
4,Integrated Methane Inversion,471626329343
...,...,...
4287,GCClassic,165131352080
4288,GCClassic,165131352080
4289,GCClassic,165131352080
4290,GCClassic,272137345468
